<a href="https://colab.research.google.com/github/SummerProjectTEAM-10/Diabets-Prediction/blob/SREVARDHANI/IQR_%2B_Label_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
from google.colab import drive
drive.mount('/content/pro')

Drive already mounted at /content/pro; to attempt to forcibly remount, call drive.mount("/content/pro", force_remount=True).


In [82]:
#Installation of required libraries
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [83]:
#Reading the dataset
df = pd.read_csv("/content/pro/MyDrive/pima-indians-diabetes.csv")

In [84]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [85]:
def median_target(var):   
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp

In [86]:
# The values to be given for incomplete observations are given the median value of people who are not sick and the median values of people who are sick.
columns = df.columns
columns = columns.drop("Outcome")
for i in columns:
    median_target(i)
    df.loc[(df['Outcome'] == 0 ) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome'] == 1 ) & (df[i].isnull()), i] = median_target(i)[i][1]

In [87]:
#outlier remove
#IQR Method

Q1=df.quantile(0.25)
Q3=df.quantile(0.75)
IQR=Q3-Q1
#outlier remove
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

(590, 9)

In [88]:
# According to BMI, some ranges are determined and categorical variables are assigned.
NewBMI = pd.Series(["Underweight", "Healthy", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
df["NewBMI"] = NewBMI
df.loc[df["BMI"] < 18.5, "NewBMI"] = NewBMI[0]
df.loc[(df["BMI"] > 18.5) & (df["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df.loc[(df["BMI"] > 24.9) & (df["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df.loc[(df["BMI"] > 29.9) & (df["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"] > 34.9) & (df["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df.loc[df["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]
# A categorical variable creation process is performed according to the insulin value.
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"
df = df.assign(NewInsulinScore=df.apply(set_insulin, axis=1))
# Some intervals were determined according to the glucose variable and these were assigned categorical variables.
NewGlucose = pd.Series(["Low", "Normal", "Prediabetes", "High"], dtype = "category")
df["NewGlucose"] = NewGlucose
df.loc[df["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df.loc[(df["Glucose"] > 70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df.loc[(df["Glucose"] > 99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df.loc[df["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore,NewGlucose
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,Obesity 1,Abnormal,High
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,Overweight,Normal,Normal
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,Healthy,Abnormal,High
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,Overweight,Normal,Normal
5,5,116.0,74.0,27.0,102.5,25.6,0.201,30,0,Overweight,Normal,Prediabetes


In [89]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
Le=LabelEncoder()
df['BMI_Lable'] = Le.fit_transform(df['NewBMI'])
df['Insulin_Lable'] = Le.fit_transform(df['NewInsulinScore'])
df['Glucose_Lable'] = Le.fit_transform(df['NewGlucose'])

In [90]:
df = df.drop(["NewBMI",	"NewInsulinScore", "NewGlucose"],axis=1)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BMI_Lable,Insulin_Lable,Glucose_Lable
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,1,0,0
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,4,1,2
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,0,0,0
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,4,1,2
5,5,116.0,74.0,27.0,102.5,25.6,0.201,30,0,4,1,3


In [91]:
# Loading train_test_split for training and testing the dataset
from sklearn.model_selection import train_test_split

In [92]:
# Train and test model
Features_Columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','BMI_Lable','Insulin_Lable','Glucose_Lable']
#Features_Columns = ['Pregnancies','Glucose','Insulin','BMI','DiabetesPedigreeFunction','Age','BMI_Lable','Insulin_Lable','Glucose_Lable']
X = df[Features_Columns]  # INDEPENDENT VARIABLES
y = df["Outcome"]         # DEPENDENT VARIABLE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [93]:
Random_Forest = RandomForestClassifier()
Random_Forest.fit(X_train , y_train)
y_pred_Random_Forest = Random_Forest.predict(X_test)


accuracy1_RF = Random_Forest.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_RF * 100, "%")
accuracy2_RF = Random_Forest.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_RF * 100, "%")

Accuracy of train data =  100.0 %
Accuracy of test data =  88.70056497175142 %
